In [1]:
%pip install pandas numpy matplotlib seaborn folium haversine scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster, HeatMap
from sklearn.cluster import DBSCAN
from datetime import datetime
import matplotlib.font_manager as fm
import os
from haversine import haversine
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 한글 폰트 설정 - Windows 환경
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 데이터 로드
try:
    reports = pd.read_csv('reports.csv', encoding='utf-8')
    parks = pd.read_csv('parks.csv', encoding='utf-8')
    print("데이터 로드 성공!")
except Exception as e:
    print(f"데이터 로드 중 오류 발생: {e}")
    # CP949도 시도
    try:
        reports = pd.read_csv('reports.csv', encoding='cp949')
        parks = pd.read_csv('parks.csv', encoding='cp949')
        print("CP949 인코딩으로 데이터 로드 성공!")
    except Exception as e2:
        print(f"CP949 인코딩으로도 실패: {e2}")

데이터 로드 성공!


In [4]:
print("\n----- 민원 데이터 기본 정보 -----")
print(f"데이터 크기: {reports.shape}")
print(reports.info())
print(reports.head())


----- 민원 데이터 기본 정보 -----
데이터 크기: (3064538, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3064538 entries, 0 to 3064537
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   민원접수일   object 
 1   민원접수시간  object 
 2   주소      object 
 3   경도      float64
 4   위도      float64
 5   요일      object 
dtypes: float64(2), object(4)
memory usage: 140.3+ MB
None
        민원접수일    민원접수시간                       주소          경도         위도  \
0  2021-09-29  19:29:00      서울특별시 강서구 강서로15길 49  126.843247  37.532089   
1  2021-09-29  18:48:00        성북구 오패산로19길 34-5   127.033761  37.609537   
2  2021-09-29  18:47:00  장위로21다길 59-19 주소지 앞도로 외  127.045741  37.616406   
3  2021-09-29  18:47:00     서울특별시 강북구 오패산로30길 13  127.034685  37.613820   
4  2021-09-29  18:46:00    서울특별시 강서구 강서로18길 52-5  126.848703  37.534293   

        요일  
0  Weekday  
1  Weekday  
2  Weekday  
3  Weekday  
4  Weekday  


In [5]:
print("\n----- 주차장 데이터 기본 정보 -----")
print(f"데이터 크기: {parks.shape}")
print(parks.info())
print(parks.head())


----- 주차장 데이터 기본 정보 -----
데이터 크기: (1463, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1463 entries, 0 to 1462
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   주소      1463 non-null   object 
 1   주차장종류   1463 non-null   object 
 2   운영구분명   1463 non-null   object 
 3   총주차면    1463 non-null   float64
 4   평일유료    1463 non-null   object 
 5   토요일유료   1463 non-null   object 
 6   공휴일유료   1463 non-null   object 
 7   평일시작    1463 non-null   object 
 8   평일종료    1463 non-null   object 
 9   토요일시작   1463 non-null   object 
 10  토요일종료   1463 non-null   object 
 11  공휴일시작   1463 non-null   object 
 12  공휴일종료   1463 non-null   object 
 13  기본주차요금  1463 non-null   float64
 14  기본주차시간  1463 non-null   float64
 15  추가단위요금  1463 non-null   float64
 16  추가단위시간  1463 non-null   float64
 17  경도      1463 non-null   float64
 18  위도      1463 non-null   float64
 19  1시간 요금  1463 non-null   float64
dtypes: float64(8), object(12)
me

In [6]:
def preprocess_data():
    # 민원 데이터 전처리
    # 날짜, 시간 형식 변환
    reports['민원접수일시'] = pd.to_datetime(reports['민원접수일'] + ' ' + reports['민원접수시간'])
    reports['월'] = reports['민원접수일시'].dt.month
    reports['일'] = reports['민원접수일시'].dt.day
    reports['시간'] = reports['민원접수일시'].dt.hour
    
    # 주차장 데이터 전처리
    # 요금 관련 컬럼 숫자로 변환
    for col in ['기본주차요금', '기본주차시간', '추가단위요금', '추가단위시간', '1시간 요금', '총주차면']:
        if col in parks.columns:
            parks[col] = pd.to_numeric(parks[col], errors='coerce')
    
    # 운영 시간 전처리
    time_cols = ['평일시작', '평일종료', '토요일시작', '토요일종료', '공휴일시작', '공휴일종료']
    for col in time_cols:
        if col in parks.columns:
            # 시간 형식 통일 (예: '0900' -> '09:00')
            parks[col] = parks[col].astype(str)
            parks[col] = parks[col].apply(lambda x: 
                                      x.zfill(4) if x.isdigit() and len(x) <= 4 
                                      else x)
            parks[col] = parks[col].apply(lambda x: 
                                      f"{x[:2]}:{x[2:]}" if x.isdigit() and len(x) == 4 
                                      else x)
    
    # 유료 여부 이진화
    binary_cols = ['평일유료', '토요일유료', '공휴일유료']
    for col in binary_cols:
        if col in parks.columns:
            parks[col] = parks[col].map({'Y': 1, 'N': 0})
    
    return reports, parks

reports, parks = preprocess_data()
print("데이터 전처리 완료!")

데이터 전처리 완료!


In [7]:
# 기본 통계적 분석
def basic_statistics():
    print("\n----- 기본 통계 분석 -----")
    
    # 민원 데이터 통계
    print("\n민원 데이터 통계:")
    print(f"총 민원 수: {len(reports)}")
    
    reports_per_day = reports.groupby('민원접수일').size()
    print(f"일평균 민원 수: {reports_per_day.mean():.2f}")
    print(f"최대 민원 발생일: {reports_per_day.idxmax()}, 건수: {reports_per_day.max()}")
    
    reports_per_weekday = reports.groupby('요일').size()
    print("\n요일별 민원 발생 건수:")
    print(reports_per_weekday)
    
    # 주차장 데이터 통계
    print("\n주차장 데이터 통계:")
    print(f"총 주차장 수: {len(parks)}")
    
    if '주차장종류' in parks.columns:
        type_counts = parks['주차장종류'].value_counts()
        print("\n주차장 종류별 수:")
        print(type_counts)
    
    if '총주차면' in parks.columns:
        print(f"\n총 주차면 수: {parks['총주차면'].sum()}")
        print(f"주차장당 평균 주차면 수: {parks['총주차면'].mean():.2f}")
    
    if '1시간 요금' in parks.columns:
        print(f"\n평균 1시간 주차 요금: {parks['1시간 요금'].mean():.2f}원")
        print(f"최소 1시간 주차 요금: {parks['1시간 요금'].min()}원")
        print(f"최대 1시간 주차 요금: {parks['1시간 요금'].max()}원")

basic_statistics()


----- 기본 통계 분석 -----

민원 데이터 통계:
총 민원 수: 3064538
일평균 민원 수: 3427.89
최대 민원 발생일: 2023-04-23, 건수: 9669

요일별 민원 발생 건수:
요일
Holiday      546988
Saturday     490527
Weekday     2027023
dtype: int64

주차장 데이터 통계:
총 주차장 수: 1463

주차장 종류별 수:
주차장종류
NS    757
NW    706
Name: count, dtype: int64

총 주차면 수: 82780.0
주차장당 평균 주차면 수: 56.58

평균 1시간 주차 요금: 1985.55원
최소 1시간 주차 요금: 0.0원
최대 1시간 주차 요금: 9200.0원


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import numpy as np

def temporal_analysis(reports: pd.DataFrame, prefix: str = 'temporal_analysis'):
    # 폰트 설정
    plt.rcParams['font.family'] = 'Malgun Gothic'
    plt.rcParams['axes.unicode_minus'] = False

    df = reports.copy()

    # 접수일시 생성
    if '시간' in df.columns:
        df['접수일시'] = pd.to_datetime(df['민원접수일'].astype(str) + ' ' + df['시간'].astype(str), errors='coerce')
    else:
        df['접수일시'] = pd.to_datetime(df['민원접수일'], errors='coerce')

    df = df.dropna(subset=['접수일시'])  # 잘못된 시간 제거

    # 시간 관련 파생 변수 생성
    dow_map = {0: '월요일', 1: '화요일', 2: '수요일', 3: '목요일', 4: '금요일', 5: '토요일', 6: '일요일'}
    df['요일'] = df['접수일시'].dt.dayofweek.map(dow_map)
    df['시간대'] = df['접수일시'].dt.hour
    df['월'] = df['접수일시'].dt.month
    df['연도'] = df['접수일시'].dt.year

    # 1) 요일별 - 각 요일별로 뚜렷한 색상 구분
    weekday_colors = {
        '월요일': '#FF9999',    
        '화요일': '#FFCC99',   
        '수요일': '#FFFF99',    
        '목요일': '#99FF99',   
        '금요일': '#99CCFF',   
        '토요일': '#9999FF',    
        '일요일': '#FF99FF'    
    }

    weekday_order = list(weekday_colors.keys())
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x='요일', order=weekday_order, palette=weekday_colors)
    plt.title('요일별 민원 발생 건수', fontsize=16, fontweight='bold')
    plt.xlabel('요일', fontsize=12)
    plt.ylabel('건수', fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{prefix}_weekday.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 2) 시간대별
    time_colors = (
        ['#FFDAB9'] * 6 +      # 0-5시: 새벽 
        ['#FFA07A'] * 6 +      # 6-11시: 오전 
        ['#FF6347'] * 6 +      # 12-17시: 오후 
        ['#8B0000'] * 6        # 18-23시: 저녁 
    )
    plt.figure(figsize=(12, 6))
    ax = sns.countplot(data=df, x='시간대', order=list(range(24)), palette=time_colors)
    plt.title('시간대별 민원 발생 건수', fontsize=16, fontweight='bold')
    plt.xlabel('시간 (시)', fontsize=12)
    plt.ylabel('건수', fontsize=12)

    
    plt.tight_layout()
    plt.savefig(f'{prefix}_hour.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 3) 월별 - 계절별 색상 (봄/여름/가을/겨울)
    season_colors = {
        12: '#EC7063',  
        1: '#EC7063',    
        2: '#EC7063',   
        
        3: '#EC7063',   
        4: '#F7DC6F',  
        5: '#F7DC6F',  
        
        6: '#F7DC6F',  
        7: '#F7DC6F',   
        8: '#F7DC6F',  
        
        9: '#F7DC6F',   
        10: '#EC7063',  
        11: '#EC7063'   
    }

    
    
    plt.figure(figsize=(10, 6))
    ax = sns.countplot(data=df, x='월', order=list(range(1, 13)), palette=[season_colors[i] for i in range(1, 13)])
    plt.title('월별 민원 발생 건수', fontsize=16, fontweight='bold')
    plt.xlabel('월', fontsize=12)
    plt.ylabel('건수', fontsize=12)
    
    plt.tight_layout()
    plt.savefig(f'{prefix}_month.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 4) 일별 민원 발생 추이 - 연도별로 뚜렷한 색상 구분
    df_daily = df.set_index('접수일시').resample('D').size().reset_index(name='건수')
    df_daily['연도'] = df_daily['접수일시'].dt.year
    
    # 연도별 색상 팔레트 (최대 10년까지 대응)
    year_colors = ['#80DEEA', '#B39DDB', '#FF8A80', '#F39C12', '#9B59B6', 
                   '#1ABC9C', '#E67E22', '#34495E', '#E91E63', '#FF5722']
    
    years = sorted(df_daily['연도'].unique())
    color_map = {year: year_colors[i % len(year_colors)] for i, year in enumerate(years)}

    plt.figure(figsize=(14, 8))
    for year, group in df_daily.groupby('연도'):
        plt.plot(group['접수일시'], group['건수'], 
                label=f'{year}년', 
                color=color_map[year],
                marker='o', 
                markersize=4,
                linewidth=2,
                alpha=0.8)

    plt.title('일별 민원 발생 추이 (연도별 비교)', fontsize=16, fontweight='bold')
    plt.xlabel('날짜', fontsize=12)
    plt.ylabel('건수', fontsize=12)
    plt.legend(title='연도', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    
    # 날짜 포맷 설정
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.xticks(rotation=45, ha='right')
    
    plt.tight_layout()
    plt.savefig(f'{prefix}_daily.png', dpi=300, bbox_inches='tight')
    plt.close()

    print("시간적 분석 완료! 생성된 파일:")
    for suffix in ['weekday', 'hour', 'month', 'daily']:
        print(f"- {prefix}_{suffix}.png")
 
# 함수 실행
temporal_analysis(reports)

시간적 분석 완료! 생성된 파일:
- temporal_analysis_weekday.png
- temporal_analysis_hour.png
- temporal_analysis_month.png
- temporal_analysis_daily.png


In [9]:
import pandas as pd

def parking_fee_size_stats(parks: pd.DataFrame):
    # 1) 유효한 값만 추출
    fee = parks['1시간 요금'].dropna()
    size = parks['총주차면'].dropna()

    # 2) 통계량 계산
    stats = pd.DataFrame({
        'min':   [fee.min(),   size.min()],
        'max':   [fee.max(),   size.max()],
        '25%':   [fee.quantile(0.25), size.quantile(0.25)],
        '50%':   [fee.quantile(0.50), size.quantile(0.50)],
        '75%':   [fee.quantile(0.75), size.quantile(0.75)],
    }, index=['1시간 요금', '총주차면'])

    return stats

# 사용 예
stats = parking_fee_size_stats(parks)
print(stats)


        min     max    25%     50%     75%
1시간 요금  0.0  9200.0  500.0  1800.0  3000.0
총주차면    1.0  1431.0    1.0    19.0    71.5


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def parking_characteristics_analysis(parks: pd.DataFrame, reports: pd.DataFrame, radius: int = 500):
    plt.rcParams['font.family'] = 'Malgun Gothic'
    plt.rcParams['axes.unicode_minus'] = False

    # 1) 유효 좌표만 필터
    valid_reports = reports.dropna(subset=['위도','경도'])
    valid_parks   = parks.dropna(subset=['위도','경도']).reset_index(drop=True)

    # 2) 보고서 좌표 배열
    rep_coords = valid_reports[['위도','경도']].to_numpy()

    # 3) haversine 함수 (벡터화)
    def haversine_array(lat, lon, coords):
        R = 6371000  # m
        φ1, λ1 = np.radians(lat), np.radians(lon)
        φ2, λ2 = np.radians(coords[:,0]), np.radians(coords[:,1])
        dφ, dλ = φ2 - φ1, λ2 - λ1
        a = np.sin(dφ/2)**2 + np.cos(φ1)*np.cos(φ2)*np.sin(dλ/2)**2
        return 2 * R * np.arcsin(np.sqrt(a))

    # 4) 민원 수 계산
    complaint_counts = []
    for _, park in valid_parks.iterrows():
        dists = haversine_array(park['위도'], park['경도'], rep_coords)
        complaint_counts.append((dists <= radius).sum())
    df = valid_parks.copy()
    df['민원 수'] = complaint_counts

    # 5) 주차장 종류별 평균 민원 수
    if '주차장종류' in df:
        plt.figure(figsize=(12,6))
        order = df.groupby('주차장종류')['민원 수'].mean().sort_values(ascending=False).index
        palette = ['#FF8A80', '#80DEEA']
        sns.barplot(data=df, x='주차장종류', y='민원 수', order=order, palette=palette)
        plt.title(f'주차장 종류별 {radius}m 반경 내 평균 민원 수', fontsize=16, fontweight='bold')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('parking_complaints_type.png', dpi=300)
        plt.close()
       

    print("분석 완료! 생성된 파일:")
    print(" - parking_complaints_type.png")

parking_characteristics_analysis(parks, reports)


분석 완료! 생성된 파일:
 - parking_complaints_type.png


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def plot_complaints_scatter(parks: pd.DataFrame, reports: pd.DataFrame,
                           radius: int = 500, output_prefix: str = 'scatter'):
    plt.rcParams['font.family'] = 'Malgun Gothic'
    plt.rcParams['axes.unicode_minus'] = False

    # 1) 유효 좌표만 필터링
    valid_reports = reports.dropna(subset=['위도', '경도'])
    valid_parks   = parks.dropna(subset=['위도', '경도']).reset_index(drop=True)

    # 2) 보고서 좌표 배열
    rep_coords = valid_reports[['위도', '경도']].to_numpy()

    # 3) haversine 함수 (벡터화)
    def haversine_array(lat, lon, coords):
        R = 6371000  # m
        φ1, λ1 = np.radians(lat), np.radians(lon)
        φ2, λ2 = np.radians(coords[:, 0]), np.radians(coords[:, 1])
        dφ, dλ = φ2 - φ1, λ2 - λ1
        a = np.sin(dφ/2)**2 + np.cos(φ1)*np.cos(φ2)*np.sin(dλ/2)**2
        return 2 * R * np.arcsin(np.sqrt(a))

    # 4) 각 주차장별 반경 내 민원 수 계산
    complaint_counts = []
    for _, park in valid_parks.iterrows():
        dists = haversine_array(park['위도'], park['경도'], rep_coords)
        complaint_counts.append(int((dists <= radius).sum()))

    # 5) DataFrame 생성
    df = valid_parks.copy()
    df['민원 수'] = complaint_counts

    # 6) 총주차면 대비 산점도 + 회귀선
    plt.figure(figsize=(8, 6))
    sns.regplot(
        data=df,
        x='총주차면',
        y='민원 수',
        scatter_kws={'s': 20, 'alpha': 0.6},
        line_kws={'color': 'red'}
    )
    plt.title(f'총주차면 대비 민원 수 (반경 {radius}m)', fontsize=16, fontweight='bold')
    plt.xlabel('총주차면')
    plt.ylabel('민원 수')
    plt.tight_layout()
    plt.savefig(f'{output_prefix}_size.png', dpi=300)
    plt.close()

    # 7) 1시간 요금 대비 산점도 + 회귀선
    if '1시간 요금' in df.columns:
        plt.figure(figsize=(8, 6))
        sns.regplot(
            data=df,
            x='1시간 요금',
            y='민원 수',
            scatter_kws={'s': 20, 'alpha': 0.6},
            line_kws={'color': 'red'}
        )
        plt.title(f'1시간 요금 대비 민원 수 (반경 {radius}m)', fontsize=16, fontweight='bold')
        plt.xlabel('1시간 요금 (원)')
        plt.ylabel('민원 수')
        plt.tight_layout()
        plt.savefig(f'{output_prefix}_fee.png', dpi=300)
        plt.close()

    print("산점도 플롯이 생성되었습니다:")
    print(f" - {output_prefix}_size.png")
    if '1시간 요금' in df.columns:
        print(f" - {output_prefix}_fee.png")

plot_complaints_scatter(parks, reports, radius=500, output_prefix='parking_complaints')


산점도 플롯이 생성되었습니다:
 - parking_complaints_size.png
 - parking_complaints_fee.png
